<a href="https://colab.research.google.com/github/iRahulPandey/medium-articles/blob/master/Bayesian_Search_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a continuation of Grid & Random Search optimization

# Bayesian

The problem with grid and random search is that these are uninformed methods like bagging.

The concept of Bayesian optimization is to choose the next hyperparameter values based on the previous results. This allows the algorithm to spend more time evaluating promising hyperparameter values and less time in low-scoring regions of the hyperparameter space.

Fake data generation

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression 
import matplotlib.pyplot as plt
import seaborn as sns

# create dataset
X, y = make_regression(n_samples=20000, n_features=10, noise=0.5)

# create dataframe
df = pd.DataFrame(data=X)
df['Target'] = y

In [ ]:
# create features and target
features = df.drop(columns=['Target'])
target = df['Target']

# shape
print(features.shape)
print(target.shape)

# Split dataset
from sklearn.model_selection import train_test_split

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 5000, random_state=0)

# splitted data
print("Number of training examples "+ str(X_train.shape[0]))
print("Number of test examples "+ str(X_test.shape[0]))

(20000, 10)
(20000,)
Number of training examples 15000
Number of test examples 5000


In [ ]:
import lightgbm as lgb

# Create a training and testing dataset
train_set = lgb.Dataset(data = X_train, label = y_train)
test_set = lgb.Dataset(data = X_test, label = y_test)

In [ ]:
from hyperopt import STATUS_OK
from timeit import default_timer as timer


def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Using early stopping to find number of trees trained
    #if 'n_estimators' in hyperparameters:
        #del hyperparameters['n_estimators']
    
    # Retrieve the subsample
    subsample = hyperparameters['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type and subsample to top level keys
    hyperparameters['boosting_type'] = hyperparameters['boosting_type']['boosting_type']
    hyperparameters['subsample'] = subsample
   
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples','n_estimators']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])

    start = timer()
    
    # n_folds cross validation
    if hyperparameters['boosting_type'] == 'dart':
      cv_results = lgb.cv(hyperparameters, train_set, num_boost_round = 10000, 
                      metrics = 'rmse', nfold = 5, seed = 0, stratified=False)
    else:  
      cv_results = lgb.cv(hyperparameters, train_set, 
                        metrics = 'rmse', nfold = 5, seed = 0, stratified=False) #remove stratified=False when training classifiers 

    run_time = timer() - start
    
    # Extract the best score
    best_score = cv_results['rmse-mean'][-1]
    #print(cv_results.cv_results_)
    std_score = cv_results['rmse-stdv'][-1]
    # Loss must be minimized
    loss = best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = len(cv_results['rmse-mean'])
    
    # Add the number of estimators to the hyperparameters
    hyperparameters['n_estimators'] = n_estimators

    # Write to the csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time, best_score])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': loss, 'std_loss': std_score, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

# Domain
In Bayesian optimization frameworks, the domian is not a discrete grid but instead has probability distributions for each hyperparameter. 

In [ ]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                            [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                             {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                             {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'is_unbalance': hp.choice('is_unbalance', [True, False]),
}

# Algorithm

In [ ]:
from hyperopt import tpe
from hyperopt import Trials

# Create the algorithm
tpe_algorithm = tpe.suggest

# Record results
trials = Trials()

In [ ]:
import csv
# Create a file and open a connection
OUT_FILE = 'bayes_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

# Write column names
headers = ['loss', 'std_loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

In [ ]:
from hyperopt import fmin

# Global variable
global  ITERATION
MAX_EVALS = 10
ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

100%|██████████| 10/10 [1:26:08<00:00, 516.85s/it, best loss: 19.0431617311527]


In [ ]:
best

{'boosting_type': 1,
 'colsample_by_tree': 0.9909414271086037,
 'dart_subsample': 0.5225163149745469,
 'is_unbalance': 1,
 'learning_rate': 0.12525505001861895,
 'min_child_samples': 95.0,
 'num_leaves': 28.0,
 'reg_alpha': 0.0059354102342819015,
 'reg_lambda': 0.7419111065986218,
 'subsample_for_bin': 180000.0}

In [ ]:
# Sort the trials with lowest loss (highest AUC) first
trials_dict = sorted(trials.results, key = lambda x: x['loss'])
trials_dict[:1]

[{'hyperparameters': {'boosting_type': 'dart',
   'colsample_bytree': 0.9909414271086037,
   'is_unbalance': False,
   'learning_rate': 0.12525505001861895,
   'min_child_samples': 95,
   'n_estimators': 10000,
   'num_leaves': 28,
   'reg_alpha': 0.0059354102342819015,
   'reg_lambda': 0.7419111065986218,
   'subsample': 0.5225163149745469,
   'subsample_for_bin': 180000},
  'iteration': 1,
  'loss': 19.0431617311527,
  'status': 'ok',
  'std_loss': 0.5262268345425978,
  'train_time': 1560.726952536}]

Bayseian optimization reaches lower RMSE faster in just 10 iterations

In [ ]:
# create a a new search space comparable to grid search and random search and run it for 60 iterations
search_space = {
    'boosting_type': hp.choice('boosting_type', 
                                            [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 0.51)}]),
    'subsample_for_bin': hp.uniform('subsample_for_bin', 20000, 20001),
    'min_child_samples': hp.uniform('min_child_samples', 20, 21),
    'num_leaves': hp.uniform('num_leaves', 2, 20),
    'n_estimators': hp.quniform('n_estimators', 1000, 10000, 1000)
}

In [ ]:
hp.quniform('n_estimators', 1000, 10000, 1000)

In [ ]:
from hyperopt import fmin

# Global variable
global  ITERATION
MAX_EVALS = 50
ITERATION = 0

# Run optimization
new_space = fmin(fn = objective, space = search_space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

  0%|          | 0/50 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



100%|██████████| 50/50 [19:39<00:00, 23.59s/it, best loss: 4.2862427492918584]


In [ ]:
# import data
df = pd.read_csv('bayes_test.csv')

In [ ]:
# create dataframe
import ast
baysian_score_df = pd.DataFrame(data=zip([d.get('n_estimators') for d in df['std_loss'].map(ast.literal_eval)], [d.get('num_leaves') for d in df['std_loss'].map(ast.literal_eval)], df['loss']), columns=['number of estimators', 'number of leaves', 'mean rmse'])

In [ ]:
import plotly.express as px

fig = px.scatter(baysian_score_df, x="number of estimators", y="number of leaves",
                 color="mean rmse", color_continuous_scale="rainbow", 
                 size="mean rmse", width=600, height=500,
                 range_color=[5,55])

# Best esitmator
fig.add_scatter(x = [10000], y=[3], mode="markers", showlegend=False, marker={'size': 20, 'color': ['Orange'], 'symbol' : 'star', 'line' : dict(
                color='black',
                width=2
            )})

# format axis
fig.update_layout(xaxis_range=[0,11000],
                                   
                  font=dict(
                  family="Arial",
                  size=16,
                  color="black")
                  
                  )

# change background
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
#'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

# Change grid lines color
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='black')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='black')
fig.show()

In [ ]:
baysian_score_df.to_csv('bayesian_output.csv')

In [ ]:
df


,loss,std_loss,hyperparameters,iteration,runtime,score
0,6.932728,"{'boosting_type': 'gbdt', 'min_child_samples':...",1,31.976020,6.932728,NaN
1,6.629764,"{'boosting_type': 'gbdt', 'min_child_samples':...",2,37.407921,6.629764,NaN
2,8.129788,"{'boosting_type': 'gbdt', 'min_child_samples':...",3,10.945550,8.129788,NaN
3,5.362806,"{'boosting_type': 'gbdt', 'min_child_samples':...",4,17.111979,5.362806,NaN
4,5.199180,"{'boosting_type': 'gbdt', 'min_child_samples':...",5,25.423809,5.199180,NaN
5,5.362806,"{'boosting_type': 'gbdt', 'min_child_samples':...",6,16.968794,5.362806,NaN
6,6.033720,"{'boosting_type': 'gbdt', 'min_child_samples':...",7,29.993399,6.033720,NaN
7,8.340787,"{'boosting_type': 'gbdt', 'min_child_samples':...",8,11.800981,8.340787,NaN
8,7.723288,"{'boosting_type': 'gbdt', 'min_child_samples':...",9,43.765498,7.723288,NaN
9,6.137738,"{'boosting_type': 'gbdt', 'min_child_samples':...",10,20.334438,6.137738,NaN
